# Naïve Bayes from Scratch! 


# Outcome of this Tutorial - A Hands-On Scikit-learn Implementation of NB 
A complete walk-through of NB implementation of NB using Python's Holy Grail of Machine Learning - Scikit-learn


Let's begin with a few imports...

In [1]:
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re 

Lets first write a handy text preprocessing function 

In [2]:
def preprocess_string(str_arg):
    
    """"
        Parameters:
        ----------
        str_arg: example string to be preprocessed
        
        What the function does?
        -----------------------
        Preprocess the string argument - str_arg - such that :
        1. everything apart from letters is excluded
        2. multiple spaces are replaced by single space
        3. str_arg is converted to lower case 
        
        Example:
        --------
        Input :  Menu is absolutely perfect,loved it!
        Output:  ['menu', 'is', 'absolutely', 'perfect', 'loved', 'it']
        

        Returns:
        ---------
        Preprocessed string 
        
    """
    
    cleaned_str=re.sub('[^a-z\s]+',' ',str_arg,flags=re.IGNORECASE) #every char except alphabets is replaced
    cleaned_str=re.sub('(\s+)',' ',cleaned_str) #multiple spaces are replaced by single space
    cleaned_str=cleaned_str.lower() #converting the cleaned string to lower case
    
    return cleaned_str # returning the preprocessed string in tokenized form

# Loading the 20 newsgroup Dataset  


In [5]:
from sklearn.datasets import fetch_20newsgroups

######################### Loading Training Dataset ############################

categories=['rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey','sci.crypt'] 
newsgroups_train=fetch_20newsgroups(subset='train',categories=categories)

train_data=newsgroups_train.data #getting all training examples
train_labels=newsgroups_train.target #getting training labels

print ("Total Number of Training Examples: ",len(train_data))
print ("Total Number of Training Labels: ",len(train_labels))


Total Number of Training Examples:  2984
Total Number of Training Labels:  2984


## Here is what the training dataset looks like in its raw form .....  🤔
Training Examples : <br>
    The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics 
    
Training Labels : <br>
    Training Labels are ['rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey','sci.crypt'] - where each training     label has its own unique integer id

In [6]:
pd.DataFrame(data=np.column_stack([train_data,train_labels]),columns=["Training Examples","Training Labels"]).head()

,Training Examples,Training Labels
0,From: carl@lvsun.com (Carl Shapiro)\nSubject: ...,4
1,From: nmcglynn@buffalo.axion.bt.co.uk (Neil A....,3
2,From: viking@iastate.edu (Dan Sorenson)\nSubje...,1
3,From: randy@megatek.com (Randy Davis)\nSubject...,1
4,From: cbetz@radioman.cray.com (Charles Betz {...,3


## Let's begin with the preprocessing of the training dataset that includes
1. Text Cleaning
2. Creating the BoW representation of our training Dataset (would need the same for test dataset as well)

### 1. Text Cleaning

In [7]:
train_data=[preprocess_string(train_str) for train_str in train_data]
print ("Data Cleaning Done")
print ("Total Number of Training Examples: ",len(train_data))

Data Cleaning Done
Total Number of Training Examples:  2984


## Here's what the processed training dataset looks like

In [8]:
pd.DataFrame(data=np.column_stack([train_data,train_labels]),columns=["Training Examples","Training Labels"]).head()

,Training Examples,Training Labels
0,from carl lvsun com carl shapiro subject re es...,4
1,from nmcglynn buffalo axion bt co uk neil a mc...,3
2,from viking iastate edu dan sorenson subject r...,1
3,from randy megatek com randy davis subject re ...,1
4,from cbetz radioman cray com charles betz x cf...,3


### 2. Creating the BoW representation of our training Dataset (would need the same for test dataset as well)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer #simply import CountVectorizer
count_vect = CountVectorizer() #instantiate it's object
X_train_counts = count_vect.fit_transform(train_data) #builds a term-document matrix ands return it
print (X_train_counts.shape)


(2984, 31409)


## Regarding CountVectorizer - as explained on [Scikit_learn](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

What the Countvectorizer Does?
Takes in the text corpus, builds it's term document matrix (i.e BoW), and returns it

Every word is assigned a fixed unique integer id and vale of each cell of this matrix represents the word
count - BoW

So for example X_train_counts[ i , j ]- where i refers to a document which in our case each document specifies a training example and j refers to the index of a word w in it's respective document i- would return count of word j 

X_train_counts[0,12048] will retrieve the word count of word with the integer id = 12048 and domcent/example 
id 0

You can read more about Sklearn CountVectorizer at [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)

In [10]:
print (X_train_counts[0,12048])

0


In [11]:
print (X_train_counts)

  (0, 10610)	1
  (0, 3981)	2
  (0, 16400)	1
  (0, 5102)	1
  (0, 24887)	1
  (0, 26714)	1
  (0, 22501)	1
  (0, 9146)	2
  (0, 30671)	2
  (0, 5874)	1
  (0, 2475)	2
  (0, 26881)	1
  (0, 21545)	1
  (0, 11504)	1
  (0, 27684)	1
  (0, 27829)	1
  (0, 19592)	1
  (0, 15457)	1
  (0, 29617)	1
  (0, 26890)	1
  (0, 15932)	1
  (0, 13247)	2
  (0, 1470)	1
  (0, 1277)	1
  (0, 15217)	2
  :	:
  (2983, 609)	1
  (2983, 8906)	1
  (2983, 26651)	1
  (2983, 17403)	1
  (2983, 24762)	1
  (2983, 2379)	1
  (2983, 10449)	1
  (2983, 16212)	4
  (2983, 1032)	4
  (2983, 10208)	1
  (2983, 3775)	3
  (2983, 29531)	3
  (2983, 234)	1
  (2983, 16325)	1
  (2983, 17707)	2
  (2983, 15369)	3
  (2983, 3563)	2
  (2983, 26654)	1
  (2983, 27242)	1
  (2983, 5072)	1
  (2983, 23801)	1
  (2983, 16762)	1
  (2983, 21278)	1
  (2983, 9696)	1
  (2983, 27251)	1


# That's it!!! Let's Move to Training! ⛸⛸⛸

In [12]:
from sklearn.naive_bayes import MultinomialNB #importing the Sklearn's NB Fucntionality
clf = MultinomialNB() #simply instantiate a Multinomial Naive Bayes object
clf.fit(X_train_counts, train_labels)  #calling the fit method trains it
print ("Training Completed")

Training Completed


# So Now That We Have Trained NB Model - Let's Move to Testing! 🏄🏽🏄🏽🏄🏽

In [13]:
newsgroups_test=fetch_20newsgroups(subset='test',categories=categories) #loading test data
test_data=newsgroups_test.data #get test set examples
test_labels=newsgroups_test.target #get test set labels

print ("Number of Test Examples: ",len(test_data))
print ("Number of Test Labels: ",len(test_labels))

test_data=[preprocess_string(test_str) for test_str in test_data] #need to preporcess the test set as well!!
print ("Number of Test Examples: ",len(test_data))



Number of Test Examples:  1986
Number of Test Labels:  1986
Number of Test Examples:  1986


The same count_vect object that was instantiated for training dataset will be used for test dataset.
But remeber that we are not calling fit_transform(since we only want to transform the test data into a term-document matrix whereas fit_transform fit_transform learns the vocabulary dictionary first and then returns a term-document matrix. We are supposed to learn the vocabulary on training dataset only

fit_transform : learns the vocabulary dictionary and returns term-document matrix
transform : transforms documents to document-term matrix


In [14]:
X_test_counts=count_vect.transform(test_data) #transforms test data to numerical form
print (X_test_counts.shape)

(1986, 31409)


# Now we can test on the transformed version of test data

In [15]:
predicted=clf.predict(X_test_counts)
print ("Test Set Accuracy : ",np.sum(predicted==test_labels)/float(len(predicted))) 

Test Set Accuracy :  0.9692849949647533


### The above code can be further reduced to literally 3 lines of code by using the pipeline functionality of sklearn!

# It's truly the ML Holy Toolkit!

In [16]:
from sklearn.pipeline import Pipeline #importing the pipeline functionality


"""
    We simply build a pipeline object by specifying the pipeline actions and once that pipeline object is
    used for the trainign purpose, it will automatically perform the pipeline steps int he specified order.
    In our case, as we first want to build a CountVectorizer for the purpose of BoW, and then fit/train a 
    NB model, so in exectly the same manner, we will speicify these actions in our pipeline. 
    
    Do note that, now when calling the fit method, we will pass the original textual data as now
    the count_vect in pipeline will itself convert it to numeric form. So it's important here that we
    pass the textual data or else nasty errros will pop out. Same is the case for test data as well. No need
    to count vectorize it separately :) But we do need to preprocess the test data from cleaning point of view

"""

clf=Pipeline([('count_vect', CountVectorizer()),('clf', MultinomialNB())])
clf.fit(train_data,train_labels)  
print ("Done")

Done


In [17]:

print (len(test_data))
predicted=clf.predict(test_data)
print ("Test Set Accuracy : ",np.sum(predicted==test_labels)/float(len(predicted))) 

1986
Test Set Accuracy :  0.9692849949647533
